In [216]:
import pandas as pd

data = 'C:/Users/Andreas Wauran/PycharmProjects/IMDB_Movies/Scrape/top1000movies_raw.csv'

In [217]:
# Read CSV Data

In [218]:
df = pd.read_csv(data)
df.columns

Index(['budget', 'content_rating', 'critic_review', 'director', 'duration',
       'gross_us_canada', 'gross_worldwide', 'language', 'metascore',
       'opening_weekend_us_canada', 'origin', 'production_company', 'rating',
       'release_date', 'title', 'total_rating', 'user_review',
       'year_released'],
      dtype='object')

In [219]:
# Create New Budget Column based on Budget Column (just contains a number of budget)

In [220]:
df["new_budget"] = df["budget"].str.replace(",", "").str.extract(r'(\d+)').fillna(0)
df["new_budget"] = df["new_budget"].astype('int64')

In [221]:
# Create Sign Budget Column to split a currency code based on Budget Column

In [222]:
df["sign_budget"] = df["budget"].str.replace(",", "").str.replace("(estimated)", "").str.extract(r"(\D+)")
df["sign_budget"] = df["sign_budget"].str.replace("\xa0", "").fillna("-")

In [223]:
# Convert budget currency to dollar currency and Create a new column named budget ($)

In [224]:
convertion = {
    "$":1,
    "¥":0.0072,
    "R$":0.052292,
    "₹":0.012,
    "£":1.25,
    "A$":0.67675,
    "€":1.09,
    "CA$":0.74,
    "FRF":0.1659,
    "ESP":0.006537,
    "SEK":0.0963,
    "DEM":0.5561,
    "ITL":0.0005617,
    "DKK":0.146,
    "RUR":0.0126,
    "TRL":0.0000000508,
    "₩":0.000747,
    "-":0
}

x = []

for index, row in df.iterrows():
    code = row['sign_budget']
    nilai = row['new_budget']
    x.append(nilai * convertion[code])
    
df["budget ($)"] = x
df["budget ($)"] = df["budget ($)"].astype("int64")

df = df.drop(["budget", "sign_budget", "new_budget"], axis=1)

In [225]:
# Convert duration Column to Minutes and Create a new column named "duration (min)"

In [226]:
duration = []

for i in df['duration']:
    if "h" in i:
        hours = int(i.split(" ")[0].replace("h", "")) * 60
    else:
        hours = 0
    
    if len(i.split(" ")) == 2: 
        minutes = int(i.split(" ")[1].replace("m", ""))
    else:
        minutes = 0
    duration.append(hours + minutes)

In [227]:
df["duration (min)"] = duration
df = df.drop(["duration"], axis=1)

In [228]:
# Convert rating type column and multipled each values with 10

In [229]:
df["rating"] = df["rating"]*10
df["rating"] = df["rating"].astype(int)

In [230]:
# Remove $ in gross_us_canada column

In [231]:
df["gross_us_canada ($)"] = df["gross_us_canada"].str.replace(",", "").str.extract(r"(\d+)").fillna(0)
df["gross_us_canada ($)"] = df['gross_us_canada ($)'].astype("int64")
df = df.drop(["gross_us_canada"], axis=1)

In [232]:
# Remove $ in gross_worldwide column

In [233]:
df["gross_worldwide ($)"] = df["gross_worldwide"].str.replace(",", "").str.extract(r"(\d+)").fillna(0)
df["gross_worldwide ($)"] = df['gross_worldwide ($)'].astype("int64")
df = df.drop(["gross_worldwide"], axis=1)

In [234]:
# Remove $ in gross_worldwide column

In [235]:
df["opening_weekend_us_canada ($)"] = df["opening_weekend_us_canada"].str.replace(",", "").str.extract(r"(\d+)").fillna(0)
df["opening_weekend_us_canada ($)"] = df['opening_weekend_us_canada ($)'].astype("int64")
df = df.drop(["opening_weekend_us_canada"], axis=1)

In [236]:
# Create Function to convert M and K in total_rating and user_review column

In [237]:
def convert_to_numeric(value):
    if value[-1] == 'K':
        return int(float(value[:-1]) * 1000)
    elif value[-1] == 'M':
        return int(float(value[:-1]) * 1000000)
    else:
        return int(value)

In [238]:
df['total_rating'] = df['total_rating'].apply(convert_to_numeric)

In [239]:
df["user_review"] = df["user_review"].apply(convert_to_numeric)

In [240]:
# Convert released date column type to date type 

In [241]:
date_list = []

for date in df["release_date"]:
    date_list.append(date.split("(")[0].strip())

df["release_date"] = date_list
df["release_date"] = pd.to_datetime(df["release_date"], format='%B %d, %Y', errors='coerce')

In [242]:
# Remove row duplicate in director column 

In [243]:
df['director'] = df['director'].fillna("").str.split(',').str[0]

In [244]:
# Reindex column dataframe and create new csv file

In [245]:
new_df = df.reindex(columns=["title", "content_rating", "year_released","director", "language", "origin", "production_company",
                         "release_date", "duration (min)", "rating", "metascore", "user_review", "critic_review",
                         "total_rating", "budget ($)", "gross_worldwide ($)", "gross_us_canada ($)",
                         "opening_weekend_us_canada ($)"])
new_df.sort_values("title")
new_df.to_csv("top1000movies_imdb.csv", index=False)